In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import sys
sys.path.append('/content/drive/My Drive/Colab Notebooks')

In [0]:
from warnings import simplefilter
simplefilter(action='ignore', category=FutureWarning)

In [0]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from hyperopt import fmin, tpe, hp, STATUS_OK
from sklearn.model_selection import cross_val_score
import time

In [0]:
train = pd.read_csv("/content/drive/My Drive/Colab Notebooks/sets_de_datos/dima_train_with_features_4.csv")

In [0]:
train.set_index('id', inplace = True)

In [0]:
TARGET = 'precio'
FEATURES = ['titulo', 'descripcion', 'tipodepropiedad', 'direccion', 'ciudad',
       'provincia', 'antiguedad', 'habitaciones', 'garages', 'banos',
       'metroscubiertos', 'metrostotales', 'idzona', 'lat', 'lng', 'fecha',
       'gimnasio', 'usosmultiples', 'piscina', 'escuelascercanas',
       'centroscomercialescercanos', 'servicios_cercanos', 'utilidades_extra',
       'tipodepropiedad_mean_antiguedad', 'tipodepropiedad_mean_habitaciones',
       'tipodepropiedad_mean_garages', 'tipodepropiedad_mean_banos',
       'tipodepropiedad_mean_metroscubiertos',
       'tipodepropiedad_mean_utilidades_extra',
       'tipodepropiedad_mean_servicios_cercanos', 'ciudad_mean_antiguedad',
       'ciudad_mean_habitaciones', 'ciudad_mean_garages', 'ciudad_mean_banos',
       'ciudad_mean_metroscubiertos', 'ciudad_mean_utilidades_extra',
       'ciudad_mean_servicios_cercanos', 'provincia_mean_antiguedad',
       'provincia_mean_habitaciones', 'provincia_mean_garages',
       'provincia_mean_banos', 'provincia_mean_metroscubiertos',
       'provincia_mean_utilidades_extra', 'provincia_mean_servicios_cercanos',
       'ciudad_mean_lat', 'ciudad_mean_lng', 'provincia_mean_lat',
       'provincia_mean_lng', 'coseno_distancia_angular_al_centro_ciudad',
       'coseno_distancia_angular_al_centro_provincia',
       'coseno_distancia_angular_al_centro_pais', 'año', 'mes', 'dia',
       'metros_no_cubiertos', 'cantidad_espacios', 'metros_x_espacio',
       'metros_x_habitaciones', 'metros_x_garages', 'metros_x_banos',
       'metros_x_utilidades_extra', 'ratio_metros_cubiertos',
       'distancia_minima_Terreno_comercial',
       'distancia_minima_Local_Comercial',
       'distancia_minima_Oficina_comercial',
       'distancia_minima_Local_en_centro_comercial',
       'distancia_minima_Bodega_comercial']

In [0]:
drop_columns_1 = ['titulo', 'descripcion', 'tipodepropiedad', 'direccion', 
                  'ciudad', 'provincia', 'fecha', 'mes', 'dia', 'escuelascercanas', 
                  'centroscomercialescercanos', 'usosmultiples',
                  'servicios_cercanos', 'piscina', 'gimnasio', 'metros_no_cubiertos',
                  'utilidades_extra', 'ratio_metros_cubiertos', 'provincia_mean_lng',
                  'distancia_minima_Terreno_comercial',
                  'distancia_minima_Local_Comercial', 'provincia_mean_garages',
                  'tipodepropiedad_mean_servicios_cercanos',
                  'provincia_mean_utilidades_extra', 'antiguedad',
                  'habitaciones']
FEATURES_1 = list(set(FEATURES) - set(drop_columns_1))

In [0]:
X = train[FEATURES_1]
y = train[TARGET]

In [0]:
K = 10
def objective(hyper_parametros):

    rf_regressor = RandomForestRegressor(**hyper_parametros)
    
    score = cross_val_score(rf_regressor, X, y, 
                            scoring='neg_mean_absolute_error', cv=10).mean()
    
    print("SCORE: {:.3f} params {}".format(score, hyper_parametros))
    
    return {'loss': -score, 'status': STATUS_OK}

In [0]:
space = {
    'max_depth': hp.randint("max_depth", 15) + 5,  
    'max_features': hp.randint("max_features", 10) + 5, 
    'n_estimators': hp.randint("n_estimators", 400) + 200,
    'min_samples_leaf': hp.randint('min_samples_leaf', 10) + 1,
    'min_samples_split' : hp.randint('min_samples_split', 10) + 2,
}

In [0]:
N_ITER = 100
t0 = time.time()
best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=N_ITER)
t1 = time.time()
print('Tiempo = {0:.2f} horas'.format((t1 - t0)/3600))

SCORE: -718602.438 params {'max_depth': 8, 'max_features': 13, 'min_samples_leaf': 3, 'min_samples_split': 7, 'n_estimators': 423}
SCORE: -827641.060 params {'max_depth': 5, 'max_features': 14, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 389}
SCORE: -722483.887 params {'max_depth': 8, 'max_features': 10, 'min_samples_leaf': 10, 'min_samples_split': 10, 'n_estimators': 299}
SCORE: -738378.851 params {'max_depth': 8, 'max_features': 5, 'min_samples_leaf': 2, 'min_samples_split': 8, 'n_estimators': 302}
SCORE: -703435.084 params {'max_depth': 9, 'max_features': 7, 'min_samples_leaf': 5, 'min_samples_split': 4, 'n_estimators': 232}
SCORE: -703731.643 params {'max_depth': 9, 'max_features': 7, 'min_samples_leaf': 10, 'min_samples_split': 7, 'n_estimators': 250}
SCORE: -639390.435 params {'max_depth': 12, 'max_features': 8, 'min_samples_leaf': 3, 'min_samples_split': 8, 'n_estimators': 520}
SCORE: -763658.722 params {'max_depth': 7, 'max_features': 6, 'min_samples_leaf': 7

In [0]:
SCORE: -718602.438 params {'max_depth': 8, 'max_features': 13, 'min_samples_leaf': 3, 'min_samples_split': 7, 'n_estimators': 423}
SCORE: -827641.060 params {'max_depth': 5, 'max_features': 14, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 389}
SCORE: -722483.887 params {'max_depth': 8, 'max_features': 10, 'min_samples_leaf': 10, 'min_samples_split': 10, 'n_estimators': 299}
SCORE: -738378.851 params {'max_depth': 8, 'max_features': 5, 'min_samples_leaf': 2, 'min_samples_split': 8, 'n_estimators': 302}
SCORE: -703435.084 params {'max_depth': 9, 'max_features': 7, 'min_samples_leaf': 5, 'min_samples_split': 4, 'n_estimators': 232}
SCORE: -703731.643 params {'max_depth': 9, 'max_features': 7, 'min_samples_leaf': 10, 'min_samples_split': 7, 'n_estimators': 250}
SCORE: -639390.435 params {'max_depth': 12, 'max_features': 8, 'min_samples_leaf': 3, 'min_samples_split': 8, 'n_estimators': 520}
SCORE: -763658.722 params {'max_depth': 7, 'max_features': 6, 'min_samples_leaf': 7, 'min_samples_split': 2, 'n_estimators': 354}
SCORE: -584129.352 params {'max_depth': 17, 'max_features': 13, 'min_samples_leaf': 8, 'min_samples_split': 5, 'n_estimators': 456}
SCORE: -690877.390 params {'max_depth': 9, 'max_features': 14, 'min_samples_leaf': 8, 'min_samples_split': 9, 'n_estimators': 332}
SCORE: -588346.328 params {'max_depth': 17, 'max_features': 10, 'min_samples_leaf': 8, 'min_samples_split': 3, 'n_estimators': 214}
SCORE: -594802.270 params {'max_depth': 15, 'max_features': 12, 'min_samples_leaf': 6, 'min_samples_split': 10, 'n_estimators': 480}
SCORE: -790487.664 params {'max_depth': 6, 'max_features': 9, 'min_samples_leaf': 7, 'min_samples_split': 6, 'n_estimators': 496}
SCORE: -748579.612 params {'max_depth': 7, 'max_features': 14, 'min_samples_leaf': 6, 'min_samples_split': 5, 'n_estimators': 352}
SCORE: -622046.260 params {'max_depth': 13, 'max_features': 9, 'min_samples_leaf': 6, 'min_samples_split': 6, 'n_estimators': 300}
SCORE: -828315.773 params {'max_depth': 5, 'max_features': 13, 'min_samples_leaf': 3, 'min_samples_split': 10, 'n_estimators': 207}
SCORE: -827619.816 params {'max_depth': 5, 'max_features': 14, 'min_samples_leaf': 9, 'min_samples_split': 11, 'n_estimators': 583}

SCORE: -718602.438 params {'max_depth': 8, 'max_features': 13, 'min_samples_leaf': 3, 'min_samples_split': 7, 'n_estimators': 423}
SCORE: -827641.060 params {'max_depth': 5, 'max_features': 14, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 389}
SCORE: -722483.887 params {'max_depth': 8, 'max_features': 10, 'min_samples_leaf': 10, 'min_samples_split': 10, 'n_estimators': 299}
SCORE: -738378.851 params {'max_depth': 8, 'max_features': 5, 'min_samples_leaf': 2, 'min_samples_split': 8, 'n_estimators': 302}
SCORE: -703435.084 params {'max_depth': 9, 'max_features': 7, 'min_samples_leaf': 5, 'min_samples_split': 4, 'n_estimators': 232}
SCORE: -703731.643 params {'max_depth': 9, 'max_features': 7, 'min_samples_leaf': 10, 'min_samples_split': 7, 'n_estimators': 250}
SCORE: -639390.435 params {'max_depth': 12, 'max_features': 8, 'min_samples_leaf': 3, 'min_samples_split': 8, 'n_estimators': 520}
SCORE: -763658.722 params {'max_depth': 7, 'max_features': 6, 'min_samples_leaf': 7, 'min_samples_split': 2, 'n_estimators': 354}
SCORE: -584129.352 params {'max_depth': 17, 'max_features': 13, 'min_samples_leaf': 8, 'min_samples_split': 5, 'n_estimators': 456}
SCORE: -690877.390 params {'max_depth': 9, 'max_features': 14, 'min_samples_leaf': 8, 'min_samples_split': 9, 'n_estimators': 332}
SCORE: -588346.328 params {'max_depth': 17, 'max_features': 10, 'min_samples_leaf': 8, 'min_samples_split': 3, 'n_estimators': 214}
SCORE: -594802.270 params {'max_depth': 15, 'max_features': 12, 'min_samples_leaf': 6, 'min_samples_split': 10, 'n_estimators': 480}
SCORE: -790487.664 params {'max_depth': 6, 'max_features': 9, 'min_samples_leaf': 7, 'min_samples_split': 6, 'n_estimators': 496}
SCORE: -748579.612 params {'max_depth': 7, 'max_features': 14, 'min_samples_leaf': 6, 'min_samples_split': 5, 'n_estimators': 352}
SCORE: -622046.260 params {'max_depth': 13, 'max_features': 9, 'min_samples_leaf': 6, 'min_samples_split': 6, 'n_estimators': 300}
SCORE: -828315.773 params {'max_depth': 5, 'max_features': 13, 'min_samples_leaf': 3, 'min_samples_split': 10, 'n_estimators': 207}
SCORE: -827619.816 params {'max_depth': 5, 'max_features': 14, 'min_samples_leaf': 9, 'min_samples_split': 11, 'n_estimators': 583}

In [0]:
#-584129.352 params {'max_depth': 17, 'max_features': 13, 'min_samples_leaf': 8, 'min_samples_split': 5, 'n_estimators': 456}